In [1]:
import os
import sys

WORKSPACE = os.path.abspath(os.path.pardir)
SRC_DIR = os.path.join(WORKSPACE, "src")

sys.path.insert(0, SRC_DIR)

In [2]:
from seqlogic import Module, Vec, run
from seqlogic.control.globals import drv_clock, drv_reset

from vcd import VCDWriter

In [3]:
class Top(Module):
    def build(self):
        clk = self.input(name="clk", dtype=Vec[1])
        rst = self.input(name="rst", dtype=Vec[1])

        q = self.logic(name="q", dtype=Vec[1])
        d = self.logic(name="d", dtype=Vec[1])

        self.expr(d, ~q)
        self.dff_r(q, d, clk, rst, rval="1b0")

        self.drv(drv_clock(clk))
        self.drv(drv_reset(rst))

In [4]:
with (
    open("flop.vcd", "w") as f,
    VCDWriter(f, timescale="1ns") as vcdw,
):
    top = Top(name="top")
    top.dump_vcd(vcdw, ".*")
    async def main():
        await top.elab()
    run(main(), ticks=20)